In [ ]:
# Name: Sundar Swaminathan, Course: CIS-583, UMICH_ID: 29200876

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras import Input

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


digits = load_digits()
print(digits.data)
print(digits.target)
data = np.asarray(digits.data, dtype='float32')
target = np.asarray(digits.target, dtype='float32')
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

input_dim = X_train.shape[1]
print(input_dim)
hidden_dim = 100
output_dim = Y_train.shape[1]

def build_model(optimizer):
    # model = Sequential()
    # model.add(Dense(hidden_dim, input_dim=input_dim, activation="tanh"))
    # model.add(Dense(hidden_dim, activation="tanh"))
    # model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model = Sequential()
    model.add(Dense(100, Input(input_dim), activation="tanh"))
    model.add(Dense(100, activation="tanh"))
    model.add(Dense(10, activation="softmax"))
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def train_and_plot(optimizer, lr, vs, title):
    print(f"Training with {title}, Validation Split: {vs}, Learning Rate: {lr}")
    model = build_model(optimizer)
    history = model.fit(X_train, Y_train, validation_split=vs, epochs=15, batch_size=32, verbose=0)
    history_df = pd.DataFrame(history.history)
    history_df["epoch"] = history.epoch
    fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(10, 6))
    history_df.plot(x="epoch", y=["loss", "val_loss"], ax=ax0, title=f"Loss ({title})")
    history_df.plot(x="epoch", y=["accuracy", "val_accuracy"], ax=ax1, title=f"Accuracy ({title})")
    plt.show()

# for lr in [0.01, 0.001, 1.0]:
#     for vs in [0.1, 0.2]:
train_and_plot(optimizers.SGD(learning_rate=0.1), 0.1, 0.2, f"SGD with LR={lr}")

# train_and_plot(optimizers.Adam(), "default", 0.2, "Adam Optimizer")

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
[0 1 2 ... 8 9 8]
Training with SGD with LR=1.0, Validation Split: 0.2, Learning Rate: 0.1


ValueError: Cannot convert '64' to a shape.